In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('cardekho_imputated.csv',index_col=False)
df.head()

,Unnamed: 0,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [4]:
df.drop(columns=['Unnamed: 0','car_name','brand'],inplace=True,axis=1)

In [5]:
df.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [6]:
x = df.drop(columns=['selling_price'],axis =1)
y = df['selling_price']

In [7]:
x

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5
1,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5
2,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5
3,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5
4,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5
...,...,...,...,...,...,...,...,...,...,...
15406,i10,9,10723,Dealer,Petrol,Manual,19.81,1086,68.05,5
15407,Ertiga,2,18000,Dealer,Petrol,Manual,17.50,1373,91.10,7
15408,Rapid,6,67000,Dealer,Diesel,Manual,21.14,1498,103.52,5
15409,XUV500,5,3800000,Dealer,Diesel,Manual,16.00,2179,140.00,7


In [8]:
from sklearn.preprocessing import LabelEncoder
l1 = LabelEncoder()

In [9]:
x['model'] = l1.fit_transform(x['model'])

In [10]:
x.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats
0,7,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5
1,54,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5
2,118,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5
3,7,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5
4,38,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5


In [11]:
len(x['transmission_type'].value_counts()),len(x['seller_type'].value_counts()),len(x['fuel_type'].value_counts())

(2, 3, 5)

In [12]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15411 entries, 0 to 15410
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   model              15411 non-null  int32  
 1   vehicle_age        15411 non-null  int64  
 2   km_driven          15411 non-null  int64  
 3   seller_type        15411 non-null  object 
 4   fuel_type          15411 non-null  object 
 5   transmission_type  15411 non-null  object 
 6   mileage            15411 non-null  float64
 7   engine             15411 non-null  int64  
 8   max_power          15411 non-null  float64
 9   seats              15411 non-null  int64  
dtypes: float64(2), int32(1), int64(4), object(3)
memory usage: 1.1+ MB


In [13]:
numerical_fe = [feature for feature in df.columns if df[feature].dtype != 'O']
print("number of numerical feature is ",len(numerical_fe))
categorical_fe = [feature for feature in df.columns if df[feature].dtype == 'O']
print("number of categorical feature is ",len(categorical_fe))
discrete_fe = [feature for feature in numerical_fe if len(df[feature].unique())<= 25]
print("number of discrete feature is ",len(discrete_fe))
continuous_fe = [feature for feature in numerical_fe if feature not in discrete_fe]
print("number of continous feature is ",len(continuous_fe))

number of numerical feature is  7
number of categorical feature is  4
number of discrete feature is  2
number of continous feature is  5


In [14]:
num_feature = x.select_dtypes(exclude='object').columns
one_hot_feature = ['transmission_type','seller_type','fuel_type']

In [15]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder(drop='first')

preprocessor = ColumnTransformer(
    [
         ("OneHotEncoder", oh_transformer, one_hot_feature),
          ("StandardScaler", numeric_transformer, num_feature)
    ]
)

In [16]:
preprocessor

ColumnTransformer(transformers=[('OneHotEncoder', OneHotEncoder(drop='first'),
                                 ['transmission_type', 'seller_type',
                                  'fuel_type']),
                                ('StandardScaler', StandardScaler(),
                                 Index(['model', 'vehicle_age', 'km_driven', 'mileage', 'engine', 'max_power',
       'seats'],
      dtype='object'))])

In [17]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [18]:
x_train = preprocessor.fit_transform(x_train)

In [19]:
x_test = preprocessor.transform(x_test)

In [20]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [21]:
def evaluate(true_value,pred_value):
    mae = mean_absolute_error(true_value, pred_value)
    mse = mean_squared_error(true_value, pred_value)
    rmse = np.sqrt(mean_squared_error(true_value, pred_value))
    r2_square = r2_score(true_value, pred_value)
    return mae, rmse, r2_square

In [22]:
model = {
    "linear_regression":LinearRegression(),
    "Adaboost": AdaBoostRegressor(),
    "GradientBoostingRegressor":GradientBoostingRegressor(),
    "lasso" : Lasso(),
    "rigde" : Ridge(),
    "K nearest neighbour": KNeighborsRegressor(),
    "Decision Tree Regressor": DecisionTreeRegressor(),
    "Random Forest Regressor" : RandomForestRegressor(),
    "XGBRegressor":XGBRegressor()
}

for i in range(len(list(model))):
    models = list(model.values())[i]
    models.fit(x_train,y_train)

    y_train_pred = models.predict(x_train)
    y_test_pred = models.predict(x_test)

     # Training set performance
    model_train_rmse,model_train_mae,model_train_r2_score = evaluate(y_train, y_train_pred)
    model_test_rmse,model_test_mae,model_test_r2_score = evaluate(y_test, y_test_pred)


    print(list(model.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2_score))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2_score))
    
    print('='*35)
    print('\n')

    

linear_regression
Model performance for Training set
- Root Mean Squared Error: 268101.6071
- Mean Absolute Error: 553855.6665
- R2 Score: 0.6218
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 279618.5794
- Mean Absolute Error: 502543.5930
- R2 Score: 0.6645


Adaboost
Model performance for Training set
- Root Mean Squared Error: 362534.0980
- Mean Absolute Error: 474433.2061
- R2 Score: 0.7225
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 383444.9055
- Mean Absolute Error: 541968.2241
- R2 Score: 0.6098


GradientBoostingRegressor
Model performance for Training set
- Root Mean Squared Error: 111709.5558
- Mean Absolute Error: 204944.5104
- R2 Score: 0.9482
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 126373.9146
- Mean Absolute Error: 255559.8775
- R2 Score: 0.9132


lasso
Model performance for Training set
- Root Mean Squared Error: 268099.2286


In [23]:
# rf_params = {"max_depth": [5, 8, 15, None, 10],
#              "max_features": [5, 7, "auto", 8],
#              "min_samples_split": [2, 8, 15, 20],
#              "n_estimators": [100, 200, 500, 1000]}
# ad_params = {
#     "n_estimators":[50,60,70,80],
#     "loss":['linear','square','exponential']
# }
# gb_pramas = {
#     "loss":['squared_error', 'absolute_error', 'huber', 'quantile'],
#     "learning_rate":[0.1,0.2,0.3],
#     "n_estimators":[100,200,500],
#     "criterion":['friedman_mse', 'squared_error'],
#     "min_samples_split":[2,4,6,8],
#     "min_samples_leaf":[1,3,5,7],
#     "max_depth":[3,5,7]
# }
xg_params={
    "learning_rate":[0.1,0.01],
    "max_depth":[5,8,12,20,30],
    "n_estimators":[100,200,300],
    "colsample_bytree":[0.5,0.8,1,0.3,0.4]
}

In [24]:
random_cv_model = [
    # ("RandomForestRegressor",RandomForestRegressor(),rf_params),
    # ("AdaBoostRegressor",AdaBoostRegressor(),ad_params),
    # ("GradientBoostingRegressor",GradientBoostingRegressor(),gb_pramas)
    ("XGBRegressor",XGBRegressor(),xg_params)
]

In [25]:
from sklearn.model_selection import  RandomizedSearchCV
dict ={} 
for name,classifier,para in random_cv_model:
    random = RandomizedSearchCV(estimator=classifier,param_distributions=para,n_iter=100,refit=True,cv = 3,verbose=2,n_jobs=-1)
    random.fit(x_train,y_train)

    dict[name] = random.best_params_

for dict_name in dict:
    print(F"------------best params for {dict_name}-----------------")
    print(dict[dict_name])


Fitting 3 folds for each of 100 candidates, totalling 300 fits
------------best params for XGBRegressor-----------------
{'n_estimators': 300, 'max_depth': 8, 'learning_rate': 0.1, 'colsample_bytree': 0.5}


In [26]:
model = {
    # "K nearest neighbour": KNeighborsRegressor(n_neighbors=3,algorithm='brute',p=1),
    # "Random Forest Regressor" : RandomForestRegressor(n_estimators= 200, min_samples_split= 2, max_features= 7,max_depth= 15),
    # "AdaBoostRegressor":AdaBoostRegressor(n_estimators=60, loss='linear')
    # "GradientBoostingRegressor":GradientBoostingRegressor(n_estimators=200, min_samples_split=6, min_samples_leaf=1, max_depth=5, loss='squared_error', learning_rate=0.3, criterion='squared_error')
}

for i in range(len(list(model))):
    models = list(model.values())[i]
    models.fit(x_train,y_train)

    y_train_pred = models.predict(x_train)
    y_test_pred = models.predict(x_test)

     # Training set performance
    model_train_rmse,model_train_mae,model_train_r2_score = evaluate(y_train, y_train_pred)
    model_test_rmse,model_test_mae,model_test_r2_score = evaluate(y_test, y_test_pred)


    print(list(model.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2_score))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2_score))
    
    print('='*35)
    print('\n')

    